In [60]:
import pandas as pd
from datetime import datetime
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from slugify import slugify

In [61]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [62]:
df = pd.read_csv('csv/localities.csv', encoding='utf-8')
image = pd.read_csv('out/image.csv', encoding='utf-8')

In [196]:
newDfC = pd.DataFrame(data = df['Continent'].unique())

In [190]:
columns = ['id', 'createdDate', 'updatedDate', 'latitude', 'longitude', 'internalNote', \
          'isPopular', 'order', 'statusId', 'canShowAds', 'isActive', 'slug', \
          'mainImageId', 'coverImageId', 'type', 'countryContinentId', 'regionCountryId', \
          'regionContinentId']

In [191]:
dfCon = newDfC

In [192]:
dfCon

,0
0,South-East Asia
1,Asia
2,South America
3,Middle America
4,Africa
5,NaN


In [67]:
dfCon = pd.concat([dfCon, pd.DataFrame(columns = columns)])

In [70]:
dfCon['id'] = dfCon.index + 1
dfCon['order'] = ''
dfCon['createdDate'] = dt_string
dfCon['updatedDate'] = dt_string
dfCon['internalNote'] = ''
dfCon['isPopular'] = False
dfCon['statusId'] = 7
dfCon['canShowAds'] = False
dfCon['isActive'] = True
dfCon['type'] = 'Continent'

In [71]:
dfCon.rename(columns = {0:'name'}, inplace = True)

In [72]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="trynego")

In [73]:
def get_lat(value):
    try:
        return geolocator.geocode(str(value))[1][0]
    except:
        return ''

In [74]:
def get_lot(value):
    try:
        return geolocator.geocode(str(value))[1][1]
    except:
        return ''

In [79]:
def set_slug(value):
    return slugify(str(value))

In [80]:
dfCon['slug'] = dfCon['name'].apply(set_slug)

In [82]:
dfCon['latitude'] = dfCon['name'].apply(get_lat)
dfCon['longitude'] = dfCon['name'].apply(get_lot)

In [117]:
newDfCountries = pd.DataFrame(data = df['Country'].unique())

In [118]:
dfCountry = pd.concat([newDfCountries, pd.DataFrame(columns = columns)])

In [119]:
dfCountry['id'] = dfCountry.index + 6
dfCountry['order'] = ''
dfCountry['createdDate'] = dt_string
dfCountry['updatedDate'] = dt_string
dfCountry['internalNote'] = ''
dfCountry['isPopular'] = False
dfCountry['statusId'] = 7
dfCountry['canShowAds'] = False
dfCountry['isActive'] = True
dfCountry['type'] = 'Country'

In [120]:
dfCountry.rename(columns = {0:'name'}, inplace = True)
dfCountry['slug'] = dfCountry['name'].apply(set_slug)
dfCountry['latitude'] = dfCountry['name'].apply(get_lat)
dfCountry['longitude'] = dfCountry['name'].apply(get_lot)

In [121]:
def set_continent_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Country'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [122]:
dfCountry['countryContinentId'] = dfCountry['name'].apply(set_continent_id)

In [126]:
newDfR = pd.DataFrame(data = df['Region'].unique())

In [127]:
dfReg = pd.concat([newDfR, pd.DataFrame(columns = columns)])

In [130]:
dfReg['id'] = dfReg.index + 50
dfReg['order'] = ''
dfReg['createdDate'] = dt_string
dfReg['updatedDate'] = dt_string
dfReg['internalNote'] = ''
dfReg['isPopular'] = False
dfReg['statusId'] = 7
dfReg['canShowAds'] = False
dfReg['isActive'] = True
dfReg['type'] = 'Region'

In [131]:
dfReg.rename(columns = {0:'name'}, inplace = True)
dfReg['slug'] = dfReg['name'].apply(set_slug)
dfReg['latitude'] = dfReg['name'].apply(get_lat)
dfReg['longitude'] = dfReg['name'].apply(get_lot)

In [132]:
def set_continent_reg_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Region'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [149]:
def set_region_reg_id(value):
    try:
        return dfCountry[:][dfCountry['name'] == df[:][df['Region'] == str(value)]['Country'].values[0]]['id'].values[0]
    except:
        return ''

In [133]:
dfReg['regionContinentId'] = dfReg['name'].apply(set_continent_reg_id)

In [150]:
dfReg['regionCountryId'] = dfReg['name'].apply(set_region_reg_id)

In [160]:
def set_image_id(value):
    try:
        return image[:][image['name'] == df[:][df['Region'] == str(value)]['Photo name'].values[0]]['id'].values[0]
    except:
        return ''

In [161]:
dfReg['coverImageId'] = dfReg['name'].apply(set_image_id)

In [164]:
newDf = pd.concat([dfCon, dfCountry, dfReg])

In [171]:
dfCountry['order'] = dfCountry.index + 1

In [172]:
dfCountry.to_csv('out/countries.csv', index=False, encoding='utf-8')

In [169]:
newDf.to_csv('out/localities.csv', index=False, encoding='utf-8')